In [1]:
import soyspacing
soyspacing.__version__

'1.0.12'

In [2]:
from soyspacing.countbase import RuleDict, CountSpace
model = CountSpace()

## Training
134963_norm은 리뷰데이터에서 한글만 남겨놓은 텍스트데이터임. 이로부터 텍스트만을 가져와서 띄어쓰기 수정 모델을 학습한다. CountSpace는 텍스트만 있는 문서에서 한줄씩 텍스트를 읽으면서 띄어쓰기 패턴을 학습

In [3]:
with open('/home/paulkim/workspace/python/Korean_NLP/data/tmp/134963_norm.txt', encoding='utf-8') as f:
    for _ in range(3):
        print(next(f))

시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다

사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고

지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두ㅜㅜ최고입니다



num_lines는 fname 파일의 num_lines번째 줄까지만 학습하는 argument임. -1이면 모든 줄을 학습에 이용함. 디버깅 확인이 목적이면 적당하게 사용해주면 됨

In [4]:
model.train(fname='/home/paulkim/workspace/python/Korean_NLP/data/tmp/134963_norm.txt', num_lines=-1)

all tags length = 654328 --> 53317, (num_doc = 15602)

json형식으로 parameters를 저장하도록 json_format=True를 설정한 뒤, tmp 폴더에 모델을 저장함

In [5]:
model.save_model('./space_model.json', json_format=True)

In [6]:
!ls

Count_base_space_correction.ipynb     sentiment_vocab.pkl  space_rules.txt
Keyword_extracion_Lasso.ipynb	      sentiment_x.pkl	   vocablist
String_distance.ipynb		      sentiment_y.pkl
naver_movie_sentiment_Analysis.ipynb  space_model.json


## Test
테스트를 하기 위해 파라미터 값을 설정해보자
- arguments
    - 4개의 파라미터
        - force_abs_threshold : 점수의 절대값이 이 수준 이상이면 최고점이 아니더라도 즉각 태깅
        - nonspace_threshold : 이 점수 이하일 때만 0으로 태깅
        - space_threshold : 이 점수 이상일 때만 1로 태깅
        - min_count : L, C, R 각각의 feature 빈도수가 min_count 이하이면 불확실항 정보로 판단, 띄어스기 계산 시 무시
    - verbose : iteration마다 띄어쓰기가 어떻게 되고 있는지 확인
    - rules : 점수와 관계없이 반드시 태기을 먼저 할 (chars, tags)

In [7]:
verbose = False
mc = 10 # min_count
ft = 0.3 # force_abs_threshold
nt = -.3 # nonspace_threshold
st = 0.3 # space_threshold

before와 after를 보면 띄어씌기가 수정된 것을 볼 수 있음

In [8]:
# 기존에 학습한 데이터와 사용하고자하는 데이터가 homorgenious해야 잘 작동함
sent = "이건진짜좋은영화 라라랜드진짜좋은영화"
sent_correct, tags = model.correct(doc=sent, verbose=False, force_abs_threshold=ft, nonspace_threshold=nt, min_count=mc, space_threshold=st)

In [9]:
print("before : %s"%sent)
print("after : %s"%sent_correct)

before : 이건진짜좋은영화 라라랜드진짜좋은영화
after : 이건 진짜 좋은 영화 라라랜드진짜 좋은 영화


verbose=True이면, 각 단계별 어떻게 띄어쓰기가 수정되었는지 확인할 수 있음

이 경우는 대부분의 경우 띄어쓰기가 확실해서 force tagging이 되었음. tagg가 None으로 뜨는 경우에는, 띄어쓰기 정보가 불확실하여 띄어쓰기를 하지 않은 경우임

In [10]:
model.correct(doc=sent, verbose=True, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

Input: ? ? ? ? ? ? ? 1 ? ? ? ? ? ? ? ? ? 1 
이건진짜좋은영화 라라랜드진짜좋은영화
Force tagged (iter=1): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 
Force tagged (iter=2): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 


('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

force_abs_threshold의 값을 좀 더 키우면 forece tagging이 적게 일어남. 속도가 더 느려짐

In [11]:
model.correct(doc=sent, verbose=True, force_abs_threshold=0.9, nonspace_threshold=nt, space_threshold=st, min_count=mc)

Input: ? ? ? ? ? ? ? 1 ? ? ? ? ? ? ? ? ? 1 
이건진짜좋은영화 라라랜드진짜좋은영화
Force tagged (iter=1): 0 ? 0 ? 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Iteratively tagged (iter=1): 0 1 0 ? 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Force tagged (iter=2): 0 1 0 ? 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Iteratively tagged (iter=2): 0 1 0 1 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Force tagged (iter=3): 0 1 0 1 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Iteratively tagged (iter=3): 0 1 0 1 0 ? 0 1 0 0 0 ? ? 1 0 ? 0 1 
Force tagged (iter=4): 0 1 0 1 0 ? 0 1 0 0 0 ? ? 1 0 ? 0 1 
Iteratively tagged (iter=4): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 ? 0 1 
Force tagged (iter=5): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 ? 0 1 
Iteratively tagged (iter=5): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 
Force tagged (iter=6): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 


('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

debug=True이면 lcr점수표와 띄어스기 교정이 되는 과정이 출력됨

In [12]:
model.correct(doc=sent, debug=True, force_abs_threshold=0.9, nonspace_threshold=nt, min_count=mc, space_threshold=st)

0: 이 (-1.000, 16)	lcr = (0.000, 0.000, -1.000)
1: 건 (0.750, 24)	lcr = (0.000, 0.750, 0.000)
2: 진 (-1.000, 53)	lcr = (0.000, -1.000, -1.000)
3: 짜 (0.703, 45)	lcr = (0.750, 0.655, 0.000)
4: 좋 (-1.000, 536)	lcr = (-1.000, 0.000, -1.000)
5: 은 (0.503, 1024)	lcr = (0.000, 0.377, 0.629)
6: 영 (-1.000, 1102)	lcr = (-1.000, -1.000, -1.000)
7: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
8: 라 (-0.998, 1015)	lcr = (-1.000, -1.000, -0.995)
9: 라 (-0.938, 1430)	lcr = (-0.882, -0.962, -0.968)
10: 랜 (-1.000, 1396)	lcr = (-1.000, -1.000, 0.000)
11: 드 (0.011, 920)	lcr = (0.011, 0.000, 0.000)
12: 진 (-1.000, 29)	lcr = (0.000, 0.000, -1.000)
13: 짜 (0.655, 29)	lcr = (0.000, 0.655, 0.000)
14: 좋 (-1.000, 536)	lcr = (-1.000, 0.000, -1.000)
15: 은 (0.502, 1022)	lcr = (0.000, 0.377, 0.627)
16: 영 (-1.000, 1022)	lcr = (-1.000, -1.000, 0.000)
17: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
force tagging i=0, score=-1.000
force tagging i=2, score=-1.000
force tagging i=4, score=-1.000
force tagging i=6, score=-1.000
force ta

('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

## Load model and rules
이미 학습된 모델과 띄어쓰기가 반드시 이뤄져야 하는 rules를 아는 경우에 이를 로딩하여 이용할 수 있음

In [13]:
model = CountSpace()

model_fname = 'space_model.json'
model.load_model(model_fname, json_format=True)

아래와 같이 모델을 중복으로 로딩할 수 있음. 중복으로 로딩될 때에는 (chars, tags)의 빈도수가 추가됨

In [14]:
another_model_fname = 'space_model.json'
model.load_model(another_model_fname, json_format=True)

rules는 하나 이상의 파일이 입력될 수 있음을 고려하여 fname list로 받음
min_rule_length, max_rule_lenth는 rules에 이용되는 단어의 최소/최대 길이임

In [15]:
rules_fname = ['space_rules.txt']
min_rule_length = 2
max_rule_length = 3

rule_dict = RuleDict(min_rule_length, max_rule_length, rules_fname)

In [16]:
rule_dict.rule_dict

{'가게는': (1, 0, 0, 1),
 '가게로': (1, 0, 0, 1),
 '가게야': (1, 0, 0, 1),
 '가령': (1, 0, 1),
 '진짜': (1, 0, 1)}

In [17]:
model.correct(sent, verbose=False, min_count=10, force_abs_threshold=0.8, nonspace_threshold=0.3, space_threshold=0.3, space_importancy=1, rules=None, debug=False )

('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

In [18]:
model.correct(sent, verbose=False, force_abs_threshold=ft, min_count=mc, nonspace_threshold=nt, space_threshold=st, debug=False, rules=rule_dict)

('이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1])